In [5]:
!pip install singlestoredb
!pip install Openai

In [29]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.1 MB/s eta 0:00:00


In [30]:
import pandas as pd
import pandas as pd
import singlestoredb as db
from openai import OpenAI
import os
import PyPDF2

In [25]:
df=pd.read_csv('Resume.csv')
df.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [48]:


os.environ["OPENAI_API_KEY"]="key"
client = OpenAI()


# Initialize OpenAI API with your API key

# Function to generate embeddings using OpenAI's API
def get_embedding(text, model="text-embedding-3-small"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

def connector():
    return db.connect(host='', port='3333', user='',
                  password='', database='')



In [39]:
try:
        mydb = connector()
        mycursor = mydb.cursor()
        sql = """CREATE TABLE rvectors (
  resume_id text CHARACTER SET utf8 COLLATE utf8_general_ci,
  resume_text text CHARACTER SET utf8 COLLATE utf8_general_ci,
  vector blob,
  SORT KEY __UNORDERED ()
  , SHARD KEY ()
) AUTOSTATS_CARDINALITY_MODE=INCREMENTAL AUTOSTATS_HISTOGRAM_MODE=CREATE AUTOSTATS_SAMPLING=ON SQL_MODE='STRICT_ALL_TABLES'"""
        mycursor.execute(sql)

        # result = mycursor.fetchall()
        # for (product_name, review_title, review_text, score, vector) in result:
        #     output.append({
        #         #'product_name': product_name,
        #         'review_title': review_title,
        #         #'review_text': review_text,
        #         #'vector': vector,
        #         'score': score
        #     })
        # mydb.close()
except Exception as e:
    print(e)
    pass
finally:
    mydb.close()

In [47]:

def read_vectors(vector):
    output = []
    try:
        mydb = connector()
        mycursor = mydb.cursor()

        sql = "SELECT resume_id, resume_text, dot_product(json_array_pack(%s), vector) as score, JSON_ARRAY_UNPACK(vector) as vector FROM rvectors order by score desc limit 5"
        mycursor.execute(sql, (str(vector)))
        result = mycursor.fetchall()
        for (resume_id, resume_text, score, vector) in result:
            output.append({
                #'product_name': product_name,
                'resume_id': resume_id,
                #'resume_text':resume_text,
                #'review_text': review_text,
                #'vector': vector,
                'score': score
            })
        mydb.close()
    except Exception as e:
        print(e)
    return output

def insert_vector(resume_id, text, vector):
    id=0
    try:
        mydb = connector()
        mycursor = mydb.cursor()
        sql = 'INSERT INTO rvectors (resume_id, resume_text, vector) values (%s,%s, JSON_ARRAY_PACK(%s))'
        mycursor.executemany(sql, [(resume_id, text, str(vector))])
        id = mycursor.lastrowid
        mydb.commit()
    except Exception as e:
        print(e)
        pass
    try:
        mydb.close()
    except:
        pass
    return id
# Data Ingestion
def read_csv(file_path):
    df = pd.read_csv(file_path)
    return df[df['Category']=='INFORMATION-TECHNOLOGY']


# Manual Embedding
def create_embeddings(data):
    embeddings = []
    for _, row in data.iterrows():
        embedding = get_embedding(row['Resume_str'])
        insert_vector(row['ID'], row['Resume_str'],embedding)
        # embedding = generate_embedding(combined_text)
        # embeddings.append(embedding)
    return embeddings

def read_and_store_to_db():
    data = read_csv("Resume.csv")
    create_embeddings(data)

def search_vector_from_db():
    text= "SDLC"
    vector = get_embedding(text)
    print(read_vectors(vector))
if __name__ == "__main__":

    #Following function will read csv and store data with vectors
    #read_and_store_to_db()

    #Following function will help to search the nearest review using vectors
#     try:
#         mydb = connector()
#         mycursor = mydb.cursor()
#         sql = """CREATE TABLE myvectors (
#   product_name text CHARACTER SET utf8 COLLATE utf8_general_ci,
#   review_title text CHARACTER SET utf8 COLLATE utf8_general_ci,
#   review_text text CHARACTER SET utf8 COLLATE utf8_general_ci,
#   vector blob,
#   username text CHARACTER SET utf8 COLLATE utf8_general_ci,
#   SORT KEY __UNORDERED ()
#   , SHARD KEY ()
# ) AUTOSTATS_CARDINALITY_MODE=INCREMENTAL AUTOSTATS_HISTOGRAM_MODE=CREATE AUTOSTATS_SAMPLING=ON SQL_MODE='STRICT_ALL_TABLES'"""
#         mycursor.execute(sql)

#         # result = mycursor.fetchall()
#         # for (product_name, review_title, review_text, score, vector) in result:
#         #     output.append({
#         #         #'product_name': product_name,
#         #         'review_title': review_title,
#         #         #'review_text': review_text,
#         #         #'vector': vector,
#         #         'score': score
#         #     })
#         # mydb.close()
#     except Exception as e:
#         print(e)
#         pass
#     finally:
#         mydb.close()
    #read_and_store_to_db()
    search_vector_from_db()








[{'resume_id': '16186411', 'score': 0.4095804989337921}, {'resume_id': '17111768', 'score': 0.3942006826400757}, {'resume_id': '26746496', 'score': 0.3703523278236389}, {'resume_id': '12763627', 'score': 0.3665770888328552}, {'resume_id': '29075857', 'score': 0.3625944256782532}]
